# Conversational Flow

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Install Dependencies

Uncomment the following lines to install the required packages.

In [ ]:
# Use the animation for pip install
# loader = LoadingAnimation()
# loader.start("Installing")
# %pip install -r requirements.txt -q
# loader.stop("Installation complete")

In [ ]:
# Create reusable loading animation class
import os, sys
import time
import threading

class LoadingAnimation:
    def __init__(self):
        self.stop_event = threading.Event()
        self.animation_thread = None

    def _animate(self, message="Loading"):
        chars = "/—\\|"
        while not self.stop_event.is_set():
            for char in chars:
                sys.stdout.write('\r' + message + '... ' + char)
                sys.stdout.flush()
                time.sleep(0.1)
                if self.stop_event.is_set():
                    sys.stdout.write("\n")
                    break

    def start(self, message="Loading"):
        self.stop_event.clear()
        self.animation_thread = threading.Thread(target=self._animate, args=(message,))
        self.animation_thread.daemon = True
        self.animation_thread.start()

    def stop(self, completion_message="Complete"):
        self.stop_event.set()
        if self.animation_thread:
            self.animation_thread.join()
        print(f"\r{completion_message} ✓")

### Helper Functions

In [ ]:
import dotenv
from dotenv import dotenv_values

# Define a fake `load_dotenv` function
def _load_dotenv(*args, **kwargs):
    env_path = kwargs.get('dotenv_path', '.env')  # Default to '.env'
    parsed_env = dotenv_values(env_path)

    # Manually set valid key-value pairs
    for key, value in parsed_env.items():
        if key and value:  # Check for valid key-value pairs
            os.environ[key] = value

dotenv.load_dotenv = _load_dotenv

### Initialization and Setup
Initial imports for the CrewAI Flow and Crew and setting up the environment

In [ ]:
# Importing necessary libraries
from typing import Any, List
import time

# Importing Crew related components
from crewai import LLM

# Importing CrewAI Flow related components
from crewai.flow import Flow, listen, start, persist, or_, router
from crewai.flow.flow import FlowState

# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

Optimizing for Llama 3.3 Prompting Template

When using different models the ability to go a lower level and change the prompting template can drastically improve the performance of the model, you want to make sure to watch for the model's training prompt patterns and adjust accordingly.

For Meta's Llama you can find it [in here](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/#prompt-template)

In [ ]:
# Agents Prompting Template for Llama 3.3
system_template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{{ .System }}<|eot_id|>"""
prompt_template="""<|start_header_id|>user<|end_header_id|>{{ .Prompt }}<|eot_id|>"""
response_template="""<|start_header_id|>assistant<|end_header_id|>{{ .Response }}<|eot_id|>"""

In [ ]:
class ConversationalFlowState(FlowState):
  """
  State for the conversational flow
  """
  message: str = ""
  query_result: List[Any] = []
  conversation_history: List[Any] = []
  step_timings: dict = {}
  llm_call_time: float = 0
  search_time: float = 0

@persist()
class ConversationalFlow(Flow[ConversationalFlowState]):
  @start()
  def start_conversation(self):
    print(f"# Starting conversation\n")
    self.llm = LLM(model="groq/llama-3.3-70b-versatile")
    self.state.step_timings = {}
    self.state.llm_call_time = 0
    self.state.search_time = 0

  @router(or_('start_conversation', 'answer_user_message'))
  def listen_for_user_input(self):
    start_time = time.time()
    message = input("Enter your message: ")
    if message.lower() == "exit":
      pass
    else:
      self.state.message = message
      self.state.conversation_history.append({"role": "user", "content": message})
      self.state.step_timings['listen_for_user_input'] = time.time() - start_time
      return 'message_received'

  @router('message_received')
  def process_user_input(self):
    start_time = time.time()
    messages = self.state.conversation_history.copy()
    messages.append(
    {
      "role": "user",
      "content": """Check if you need more details about crewai enterprise features to answer.
                    Only ask for more info if the question is not clearly about crewai.

                    If you have enough info, just reply 'complete'.
                    If you need more info, reply with one search sentence.

                    Look at our chat history and my message.
                    Decide if you can give a good answer with what you know."""
    })

    llm_start = time.time()
    response = self.llm.call(messages)
    self.state.llm_call_time += time.time() - llm_start

    if response == 'complete':
      self.state.step_timings['process_user_input'] = time.time() - start_time
      return 'answer'
    else:
      # something soon
      return 'answer'


  @listen('answer')
  def answer_user_message(self):
    start_time = time.time()
    llm_start = time.time()
    response = self.llm.call(self.state.conversation_history)
    self.state.llm_call_time += time.time() - llm_start

    self.state.conversation_history.append({"role": "assistant", "content": response})
    print(f"# Assistant response: {response}\n")
    self.state.step_timings['answer_user_message'] = time.time() - start_time

    print(f"\nTiming Summary:")
    print(f"Total LLM call time: {self.state.llm_call_time:.2f}s")
    print(f"Total Search time: {self.state.search_time:.2f}s")
    print("Step timings:")
    for step, timing in self.state.step_timings.items():
        print(f"  {step}: {timing:.2f}s")


In [ ]:
flow = ConversationalFlow()
flow.kickoff()

In [ ]:
flow_new = ConversationalFlow()
flow_new.kickoff(inputs={"id": flow.flow_id})